# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="../environments/unity/Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
import os
import sys

sys.path.append(os.path.abspath('../src/'))

In [5]:
import agents

In [12]:
!pip install gym-unity

  Using cached gym_unity-0.15.1-py3-none-any.whl (7.5 kB)
  Using cached mlagents_envs-0.15.1-py3-none-any.whl (54 kB)
     |████████████████████████████████| 1.3 MB 362 kB/s eta 0:00:01
ERROR: unityagents 0.4.0 has requirement protobuf==3.5.2, but you'll have protobuf 3.11.3 which is incompatible.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.5.2
    Uninstalling protobuf-3.5.2:
      Successfully uninstalled protobuf-3.5.2


In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


In [6]:
state.dtype

dtype('float64')

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [7]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


When finished, you can close the environment.

In [8]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
def constant_annealing_schedule(n, constant):
    return constant


def exponential_annealing_schedule(n, rate):
    return 1 - np.exp(-rate * n)

In [8]:
def power_decay_schedule(episode_number: int,
                         decay_factor: float,
                         minimum_epsilon: float) -> float:
    """Power decay schedule found in other practical applications."""
    return max(decay_factor**episode_number, minimum_epsilon)

_epsilon_decay_schedule_kwargs = {
    "decay_factor": 0.99,
    "minimum_epsilon": 1e-2,
}
epsilon_decay_schedule = lambda n: power_decay_schedule(n, **_epsilon_decay_schedule_kwargs)

In [9]:
from torch import optim


_optimizer_kwargs = {
    "lr": 1e-3,
    "betas":(0.9, 0.999),
    "eps": 1e-08,
    "weight_decay": 0,
    "amsgrad": False,
}
optimizer_fn = lambda parameters: optim.Adam(parameters, **_optimizer_kwargs)

In [14]:
banana_env = UnityEnvironment(file_name="Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [15]:
brain_name = banana_env.brain_names[0]
brain = banana_env.brains[brain_name]

In [7]:
import training

In [12]:
_agent_kwargs = {
    "state_size": brain.vector_observation_space_size,
    "action_size": brain.vector_action_space_size, 
    "number_hidden_units": 64,
    "optimizer_fn": optimizer_fn,
    "epsilon_decay_schedule": epsilon_decay_schedule,
    "alpha": 0.5,
    "batch_size": 64,
    "buffer_size": 100000,
    "beta_annealing_schedule": lambda n: exponential_annealing_schedule(n, 1e-2),
    "gamma": 0.95,
    "update_frequency": 4,
    "seed": None,
}
double_dqn_agent = DeepQAgent(**_agent_kwargs)

prioritized_sampling_scores = training.train(double_dqn_agent,
                                             banana_env,
                                             brain_name,
                                             "prioritized-sampling-checkpoint.pth",
                                             number_episodes=2000,
                                             target_score=13)

Episode 100	Average Score: 0.15
Episode 200	Average Score: 0.12
Episode 300	Average Score: 0.62
Episode 400	Average Score: 0.46
Episode 500	Average Score: 1.32
Episode 600	Average Score: 3.49
Episode 700	Average Score: 6.65
Episode 800	Average Score: 9.41
Episode 900	Average Score: 10.40
Episode 1000	Average Score: 11.07
Episode 1100	Average Score: 11.94
Episode 1200	Average Score: 12.79
Episode 1300	Average Score: 11.98

Environment solved in 1372 episodes!	Average Score: 13.06


PicklingError: Can't pickle <function <lambda> at 0x7f91b08390d0>: attribute lookup <lambda> on __main__ failed

In [16]:
prioritized_sampling_scores = train(double_dqn_agent,
                                    banana_env,
                                    brain_name,
                                    "prioritized-sampling-checkpoint.pth",
                                    number_episodes=2000,
                                    target_score=15)

Episode 100	Average Score: 13.59
Episode 200	Average Score: 12.41
Episode 300	Average Score: 13.06
Episode 400	Average Score: 12.33
Episode 500	Average Score: 12.90
Episode 600	Average Score: 12.66
Episode 700	Average Score: 12.44
Episode 800	Average Score: 12.45
Episode 900	Average Score: 13.58
Episode 1000	Average Score: 11.96
Episode 1100	Average Score: 13.14
Episode 1200	Average Score: 12.50
Episode 1300	Average Score: 12.34
Episode 1400	Average Score: 12.71
Episode 1500	Average Score: 11.97
Episode 1600	Average Score: 13.47
Episode 1700	Average Score: 12.36
Episode 1800	Average Score: 12.00
Episode 1900	Average Score: 12.07
Episode 2000	Average Score: 13.02
